In [1]:
from market_growth_analysis.web_scraping import process_sector, scrape_company_data
import concurrent
import tqdm

from bs4 import BeautifulSoup
import requests

In [2]:
URL = "https://www.macrotrends.net/stocks/research"
page = requests.get(URL)


soup = BeautifulSoup(page.content, "html.parser")

results = soup.find_all("a", href= lambda text: "/stocks/sector/" in text.lower())
sector_dict = dict()

for result in results:
    sector_dict[result.text] = ("".join(("https://www.macrotrends.net", result["href"]))).strip()

with concurrent.futures.ThreadPoolExecutor() as executor:
    print("Retrieving income statement")
    financial_sheet = "income-statement"
    for sector in tqdm.tqdm(sector_dict, desc="Sectors"):
        process_sector(sector, sector_dict, financial_sheet, executor)

    print("Retrieving cash flow statements")
    financial_sheet = "cash-flow-statement"
    for sector in tqdm.tqdm(sector_dict, desc="Sectors"):
        process_sector(sector, sector_dict, financial_sheet, executor)

    print("Retrieving balance sheet")
    financial_sheet = "balance-sheet"
    for sector in tqdm.tqdm(sector_dict, desc="Sectors"):
        process_sector(sector, sector_dict, financial_sheet, executor)

Retrieving income statement


Sectors:   0%|          | 0/17 [00:00<?, ?it/s]ERROR:root:Error occurred for BRK.A: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for BRK.B: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for IWM: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for AGG: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for EFA: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for DIA: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for SPY: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for SHM: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for EEM: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for LQD: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for JNK: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for XLY: 'NoneType' object has no attribute 'groups'
E

Retrieving cash flow statements


Sectors:   0%|          | 0/17 [00:00<?, ?it/s]ERROR:root:Error occurred for BRK.A: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for BRK.B: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for SPY: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for EFA: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for AGG: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for IWM: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for DIA: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for IBB: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for EEM: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for SHM: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for XLY: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for JNK: 'NoneType' object has no attribute 'groups'
E

Retrieving balance sheet


Sectors:   0%|          | 0/17 [00:00<?, ?it/s]ERROR:root:Error occurred for BRK.B: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for BRK.A: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for LQD: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for EFA: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for IWM: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for SPY: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for AGG: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for JNK: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for SHM: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for EEM: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for XLY: 'NoneType' object has no attribute 'groups'
ERROR:root:Error occurred for IBB: 'NoneType' object has no attribute 'groups'
E